# Lab 10: LSTM Stock Prediction (cleaned)

Simplified and cleaned code from your lab manual.

In [ ]:
# Lab 10: LSTM Stock Prediction (AAPL) - cleaned
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import yfinance as yf

ticker = 'AAPL'
data = yf.download(ticker, start='2010-01-01', end='2023-01-01')
print(data.head())

data = data.sort_index()
data = data.filter(['Open'])
dataset = data.values
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
training_data_len = int(np.ceil(len(dataset) * 0.8))

train_data = scaled_data[0:training_data_len, :]
x_train = []; y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=25))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=1, epochs=1, verbose=1)

# Testing
test_data = scaled_data[training_data_len - 60:, :]
x_test = []; y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE: {rmse:.4f}')

train = data[:training_data_len]
valid = data[training_data_len:]
valid = valid.copy()
valid['Predictions'] = predictions

plt.figure(figsize=(12,6))
plt.plot(train['Open']); plt.plot(valid[['Open','Predictions']])
plt.title('Model'); plt.xlabel('Date'); plt.ylabel('Open Price USD ($)')
plt.legend(['Train','Val','Predictions'], loc='lower right'); plt.show()

print(valid.head())

In [ ]:
# End of Lab 10